In [1]:
import setup
setup.init_django()

In [2]:
from market.models import StockQuote

In [3]:
from django.db.models import (
    Avg, 
    F,
    RowRange,
    Window,
    Max,
    Min,
    ExpressionWrapper,
    DecimalField,
    Case,
    When,
    Value
)
from django.db.models.functions import TruncDate, FirstValue, Lag, Coalesce
from django.utils import timezone
from datetime import timedelta
from decimal import Decimal

In [4]:
def calculate_rsi(ticker, period=14):
    """
    Calculate Relative Strength Index (RSI) using Django ORM.
    
    Args:
        ticker (str): Stock ticker symbol
        period (int): RSI period (default: 14)
    
    Returns:
        dict: RSI value and component calculations
    """
    end_date = timezone.now()
    start_date = end_date - timedelta(days=period * 4)
    
    # Get daily price data
    daily_data = (
        StockQuote.timescale
        .filter(company__ticker=ticker, time__range=(start_date, end_date))
        .time_bucket('time', '1 day')
        .order_by('bucket')
    )
    
    # Calculate price changes and gains/losses with explicit decimal conversion
    movement = daily_data.annotate(
        closing_price=ExpressionWrapper(
            F('close_price'),
            output_field=DecimalField(max_digits=10, decimal_places=4)
        ),
        prev_close=Window(
            expression=Lag('close_price'),
            order_by=F('bucket').asc(),
            output_field=DecimalField(max_digits=10, decimal_places=4)
        )
    ).annotate(
        price_change=ExpressionWrapper(
            F('close_price') - F('prev_close'),
            output_field=DecimalField(max_digits=10, decimal_places=4)
        ),
        gain=Case(
            When(price_change__gt=0, 
                 then=ExpressionWrapper(
                     F('price_change'),
                     output_field=DecimalField(max_digits=10, decimal_places=4)
                 )),
            default=Value(0, output_field=DecimalField(max_digits=10, decimal_places=4)),
            output_field=DecimalField(max_digits=10, decimal_places=4)
        ),
        loss=Case(
            When(price_change__lt=0,
                 then=ExpressionWrapper(
                     -F('price_change'),
                     output_field=DecimalField(max_digits=10, decimal_places=4)
                 )),
            default=Value(0, output_field=DecimalField(max_digits=10, decimal_places=4)),
            output_field=DecimalField(max_digits=10, decimal_places=4)
        )
    )
    
    # Calculate initial averages for the first period
    initial_avg = movement.exclude(prev_close__isnull=True)[:period].aggregate(
        avg_gain=Coalesce(
            ExpressionWrapper(
                Avg('gain'),
                output_field=DecimalField(max_digits=10, decimal_places=4)
            ),
            Value(0, output_field=DecimalField(max_digits=10, decimal_places=4))
        ),
        avg_loss=Coalesce(
            ExpressionWrapper(
                Avg('loss'),
                output_field=DecimalField(max_digits=10, decimal_places=4)
            ),
            Value(0, output_field=DecimalField(max_digits=10, decimal_places=4))
        )
    )
    
    # Get subsequent data points for EMA calculation
    subsequent_data = list(movement.exclude(prev_close__isnull=True)[period:].values('gain', 'loss'))
    
    # Calculate EMA-based RSI
    avg_gain = initial_avg['avg_gain']
    avg_loss = initial_avg['avg_loss']
    alpha = Decimal(1 / period)  # Smoothing factor
    
    # Update moving averages using EMA formula
    for data in subsequent_data:
        avg_gain = (avg_gain * (1 - alpha) + data['gain'] * alpha)
        avg_loss = (avg_loss * (1 - alpha) + data['loss'] * alpha)
    
    # Prevent division by zero
    if avg_loss == 0:
        rsi = 100
    else:
        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))
    
    return {
        'rsi': round(float(rsi), 4),
        'avg_gain': round(float(avg_gain), 4),
        'avg_loss': round(float(avg_loss), 4),
        'period': period
    } 

In [5]:
rsi_data = calculate_rsi('AAPL')

In [6]:
rsi_data

{'rsi': 56.7483, 'avg_gain': 0.0579, 'avg_loss': 0.0441, 'period': 14}